## 基于MindSpore框架的DenseNet案例实现

### 1 模型简介

DenseNet模型于2017年在论文《Densely Connected Convolutional Networks》中被提出。DenseNet通过特征在channel上的连接来实现特征重用（feature reuse）。这些特点让DenseNet在参数和计算成本更少的情形下实现比ResNet更优的性能，DenseNet也因此斩获CVPR 2017的最佳论文奖。本篇文章首先介绍DenseNet的原理以及网路架构，然后讲解DenseNet在MindSpore上的实现。

#### 1.1 模型结构
相比ResNet，DenseNet提出了一个更激进的密集连接机制：即互相连接所有的层，具体来说就是每个层都会接受其前面所有层作为其额外的输入。图1为ResNet网络的连接机制，作为对比，图1为DenseNet的密集连接机制。可以看到，ResNet是每个层与前面的某层（一般是2~3层）短路连接在一起，连接方式是通过元素级相加。而在DenseNet中，每个层都会与前面所有层在通道维度上连接在一起，并作为下一层的输入。对于一个L层的网络，DenseNet共包含$ \frac{L(L+1)}{2}$个连接，相比ResNet，这是一种密集连接。而且DenseNet是直接连接来自不同层的特征图，这可以实现特征重用，提升效率，这一特点是DenseNet与ResNet最主要的区别。

<center>
    <img src="https://s1.ax1x.com/2022/09/24/xAeebR.jpg" alt="image-2022092401" style="zoom:75%;" />
    <br>
    <div style="color:orange;
    display: inline-block;
    color: #999;
    padding: 2px;">图1 一个5层的密集块，增长率为k=4。每一层都将所有前面的特征图作为输入。</div>
</center>

传统的网络在L层的输出为：
$$
x_{l}=H_{l}(x_{l−1})
$$

而对于ResNet，增加了来自上一层输入的identity函数：

$$
x_{l} = H_{l}(x_{l−1})+x_{l-1}
$$

在DenseNet中，会连接前面所有层作为输入：
$$
x_{l}=H_{l}([x_{0},x_{1},...,x_{l−1}])
$$

其中，上面的$H_{l}( ⋅ )$代表是非线性转化函数（non-liear transformation），它是一个组合操作，其可能包括一系列的BN(Batch Normalization)，ReLU，Pooling及Conv操作。注意这里l层与 $l−1$层之间可能实际上包含多个卷积层。

CNN网络一般要经过Pooling或者stride>1的Conv来降低特征图的大小，而DenseNet的密集连接方式需要特征图大小保持一致。为了解决这个问题，DenseNet网络中使用DenseBlock+Transition的结构，其中DenseBlock是包含很多层的模块，每个层的特征图大小相同，层与层之间采用密集连接方式。而Transition模块是连接两个相邻的DenseBlock，并且通过Pooling使特征图大小降低。图2给出了具有三个密集块的深度 DenseNet，各个DenseBlock之间通过Transition连接在一起。

<center>
    <img src="https://s1.ax1x.com/2022/09/24/xAu3xU.jpg" alt="image-2022092402" style="zoom:75%;" />
    <br>
    <div style="color:orange;
    display: inline-block;
    color: #999;
    padding: 2px;">图2 具有三个密集块的深度 DenseNet。 两个相邻块之间的层称为过渡层，并通过卷积和池化改变特征图大小。</div>
</center>


#### 1.2 DenseNet优点

a) 相比ResNet拥有更少的参数数量；

b) 旁路加强了特征的重用；

c) 网络更易于训练,并具有一定的正则效果；

d) 缓解了gradient vanishing和model degradation的问题。


#### 1.3 代码实现

昇思MindSpore联合高校打造了CV、NLP、Audio、OCR、YOLO等领域的AI套件，集成了大量主流和前沿的算法模型。DenseNet集成在MindCV中，我们通过阅读[MindCV源码](https://github.com/mindspore-lab/mindcv)来学习网络的实现。MindCV的安装请查看[这里](https://mindspore-lab.github.io/mindcv/installation/)。

- DenseLayer的构建

`growth_rate` 定义了每个 `_DenseLayer` 向网络增加的特征图（channels）的数量。在DenseNet架构中，每个密集层输出的特征图都会与之前所有层的输出特征图合并，因此，网络的特征图总数随着层数的增加而线性增长。

`bn_size` 乘以 `growth_rate` 决定了瓶颈层的大小，起到扩展特征图数量的作用，使得随后的3x3卷积层可以从更丰富的特征图中学习。

```python
class _DenseLayer(nn.Cell):
    """Basic unit of DenseBlock (using bottleneck layer)"""

    def __init__(
        self,
        num_input_features: int,
        growth_rate: int,
        bn_size: int,
        drop_rate: float,
    ) -> None:
        super().__init__()
        self.norm1 = nn.BatchNorm2d(num_input_features)
        self.relu1 = nn.ReLU()
        self.conv1 = nn.Conv2d(num_input_features, bn_size * growth_rate, kernel_size=1, stride=1)

        self.norm2 = nn.BatchNorm2d(bn_size * growth_rate)
        self.relu2 = nn.ReLU()
        self.conv2 = nn.Conv2d(bn_size * growth_rate, growth_rate, kernel_size=3, stride=1, pad_mode="pad", padding=1)

        self.drop_rate = drop_rate
        self.dropout = Dropout(p=self.drop_rate)

    def construct(self, features: Tensor) -> Tensor:
        bottleneck = self.conv1(self.relu1(self.norm1(features)))
        new_features = self.conv2(self.relu2(self.norm2(bottleneck)))
        if self.drop_rate > 0.0:
            new_features = self.dropout(new_features)
        return new_features
```

- DenseBlock的构建

`_DenseBlock` 由多个 `_DenseLayer`层组成，每个 `_DenseLayer` 生成新的特征图，然后将这些新特征图与前面层的特征图合并（使用`ops.concat`函数），实现密集连接。

```python
class _DenseBlock(nn.Cell):
    """DenseBlock. Layers within a block are densely connected."""

    def __init__(
        self,
        num_layers: int,
        num_input_features: int,
        bn_size: int,
        growth_rate: int,
        drop_rate: float,
    ) -> None:
        super().__init__()
        self.cell_list = nn.CellList()
        for i in range(num_layers):
            layer = _DenseLayer(
                num_input_features=num_input_features + i * growth_rate,
                growth_rate=growth_rate,
                bn_size=bn_size,
                drop_rate=drop_rate,
            )
            self.cell_list.append(layer)

    def construct(self, init_features: Tensor) -> Tensor:
        features = init_features
        for layer in self.cell_list:
            new_features = layer(features)
            features = ops.concat((features, new_features), axis=1)
        return features
```

- Transition层的构建

`_Transition` 层主要用于在两个相邻的DenseBlock之间进行特征图的转换和维度调整。具体地，1x1卷积用于调整特征图的深度（通道数），而平均池化则减少其空间尺寸（宽度和高度），从而在保留关键信息的同时减少后续层的计算负担。

```python
class _Transition(nn.Cell):
    """Transition layer between two adjacent DenseBlock"""

    def __init__(
        self,
        num_input_features: int,
        num_output_features: int,
    ) -> None:
        super().__init__()
        self.features = nn.SequentialCell(OrderedDict([
            ("norm", nn.BatchNorm2d(num_input_features)),
            ("relu", nn.ReLU()),
            ("conv", nn.Conv2d(num_input_features, num_output_features, kernel_size=1, stride=1)),
            ("pool", nn.AvgPool2d(kernel_size=2, stride=2))
        ]))

    def construct(self, x: Tensor) -> Tensor:
        x = self.features(x)
        return x
```

- DenseNet的构建

在DenseNet模型中，初始卷积层采用了7x7的卷积核和步长为2，负责进行初步的特征提取。随后，模型包含一系列的DenseBlock和Transition层，构成了网络的核心。最后，一个批量归一化层和ReLU激活层对网络输出进行进一步处理，为分类或其他下游任务做准备。

在`_initialize_weights` 方法中，对网络中不同类型的层进行了专门的权重初始化。对于卷积层，使用He正态初始化来适应ReLU激活函数，同时对偏置项（如果存在）应用He均匀初始化，适配Leaky ReLU非线性。批量归一化层的gamma参数被初始化为1，beta参数初始化为0，以保持其归一化功能。对于全连接层，同样使用He均匀初始化权重，并将偏置初始化为0。这种有针对性的初始化方法有助于网络在训练初期实现有效的梯度传播和快速收敛。

```python
class DenseNet(nn.Cell):
    r"""Densenet-BC model class, based on
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_

    Args:
        growth_rate: how many filters to add each layer (`k` in paper). Default: 32.
        block_config: how many layers in each pooling block. Default: (6, 12, 24, 16).
        num_init_features: number of filters in the first Conv2d. Default: 64.
        bn_size (int): multiplicative factor for number of bottleneck layers
          (i.e. bn_size * k features in the bottleneck layer). Default: 4.
        drop_rate: dropout rate after each dense layer. Default: 0.
        in_channels: number of input channels. Default: 3.
        num_classes: number of classification classes. Default: 1000.
    """

    def __init__(
        self,
        growth_rate: int = 32,
        block_config: Tuple[int, int, int, int] = (6, 12, 24, 16),
        num_init_features: int = 64,
        bn_size: int = 4,
        drop_rate: float = 0.0,
        in_channels: int = 3,
        num_classes: int = 1000,
    ) -> None:
        super().__init__()
        layers = OrderedDict()
        # first Conv2d
        num_features = num_init_features
        layers["conv0"] = nn.Conv2d(in_channels, num_features, kernel_size=7, stride=2, pad_mode="pad", padding=3)
        layers["norm0"] = nn.BatchNorm2d(num_features)
        layers["relu0"] = nn.ReLU()
        layers["pool0"] = nn.SequentialCell([
            nn.Pad(paddings=((0, 0), (0, 0), (1, 1), (1, 1)), mode="CONSTANT"),
            nn.MaxPool2d(kernel_size=3, stride=2),
        ])

        # DenseBlock
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(
                num_layers=num_layers,
                num_input_features=num_features,
                bn_size=bn_size,
                growth_rate=growth_rate,
                drop_rate=drop_rate,
            )
            layers[f"denseblock{i + 1}"] = block
            num_features += num_layers * growth_rate
            if i != len(block_config) - 1:
                transition = _Transition(num_features, num_features // 2)
                layers[f"transition{i + 1}"] = transition
                num_features = num_features // 2

        # final bn+ReLU
        layers["norm5"] = nn.BatchNorm2d(num_features)
        layers["relu5"] = nn.ReLU()

        self.num_features = num_features
        self.features = nn.SequentialCell(layers)
        self.pool = GlobalAvgPooling()
        self.classifier = nn.Dense(self.num_features, num_classes)
        self._initialize_weights()

    def _initialize_weights(self) -> None:
        """Initialize weights for cells."""
        for _, cell in self.cells_and_names():
            if isinstance(cell, nn.Conv2d):
                cell.weight.set_data(
                    init.initializer(init.HeNormal(math.sqrt(5), mode="fan_out", nonlinearity="relu"),
                                     cell.weight.shape, cell.weight.dtype))
                if cell.bias is not None:
                    cell.bias.set_data(
                        init.initializer(init.HeUniform(math.sqrt(5), mode="fan_in", nonlinearity="leaky_relu"),
                                         cell.bias.shape, cell.bias.dtype))
            elif isinstance(cell, nn.BatchNorm2d):
                cell.gamma.set_data(init.initializer("ones", cell.gamma.shape, cell.gamma.dtype))
                cell.beta.set_data(init.initializer("zeros", cell.beta.shape, cell.beta.dtype))
            elif isinstance(cell, nn.Dense):
                cell.weight.set_data(
                    init.initializer(init.HeUniform(math.sqrt(5), mode="fan_in", nonlinearity="leaky_relu"),
                                     cell.weight.shape, cell.weight.dtype))
                if cell.bias is not None:
                    cell.bias.set_data(init.initializer("zeros", cell.bias.shape, cell.bias.dtype))

    def forward_features(self, x: Tensor) -> Tensor:
        x = self.features(x)
        return x

    def forward_head(self, x: Tensor) -> Tensor:
        x = self.pool(x)
        x = self.classifier(x)
        return x

    def construct(self, x: Tensor) -> Tensor:
        x = self.forward_features(x)
        x = self.forward_head(x)
        return x
```

## 2. 案例实现

### 2.1 数据集读取
通过[mindcv.data](https://mindcv.readthedocs.io/en/latest/api/mindcv.data.html)中的[create_dataset](https://mindcv.readthedocs.io/en/latest/api/mindcv.data.html#mindcv.data.create_dataset)模块，我们可快速地读取标准数据集或自定义的数据集。

这里我们采用CIFAR-10数据集进行测试。CIFAR-10是CV领域中一个极为重要的基准数据集,它包含60,000张32x32像素的小型彩色图像，这些图像被分为10个不同的类别，包括飞机、汽车、鸟类、猫、鹿、狗、青蛙、马、船和卡车。数据集中有50,000张图像用于训练，10,000张用于测试。

In [1]:
from mindcv.data import create_dataset, create_transforms, create_loader
import os

# 数据集路径
cifar10_dir = './datasets/cifar' # 你的数据存放路径
num_classes = 10 # 类别数
num_workers = 8 # 数据读取及加载的工作线程数 
download = not os.path.exists(cifar10_dir)

# 创建数据集
dataset_train = create_dataset(name='cifar10', root=cifar10_dir, split='train', shuffle=True, num_parallel_workers=num_workers, download=download)
cifar10_dir += '/cifar-10-batches-bin'

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
170052608B [06:34, 431556.60B/s]                                 


`name`: 数据集名称，如MNIST、CIFAR10、ImageNet等。空字符串('')表示自定义数据集。默认值：''。对于自定义数据集，数据集目录应遵循以下结构：
```text
.dataset_name/
├── split1/
│  ├── class1/
│  │   ├── 000001.jpg
│  │   ├── 000002.jpg
│  │   └── ....
│  └── class2/
│      ├── 000001.jpg
│      ├── 000002.jpg
│      └── ....
└── split2/
	├── class1/
	│   ├── 000001.jpg
	│   ├── 000002.jpg
	│   └── ....
	└── class2/
		├── 000001.jpg
		├── 000002.jpg
		└── ....
```

`split`: 数据分割方式，'' 或 分割名字符串（train/val/test），如果为 ''，则不使用分割。否则，它是根目录的子文件夹，例如 train、val、test。默认值：'train'。通过调整参数，可以轻松切换训练、验证和测试数据集。

完整的参数解释可以ctrl+左键点击函数名快速访问MindCV源码获取。

### 2.2 数据处理及加载
1. 通过[create_transforms](https://mindcv.readthedocs.io/en/latest/api/mindcv.data.html#mindcv.data.create_transforms)函数, 可直接得到标准数据集合适的数据处理增强策略(transform list)，包括Cifar10, ImageNet上常用的数据处理策略。

In [2]:
# 创建所需的数据增强操作的列表
trans = create_transforms(dataset_name='cifar10', image_resize=224)

通过查看MindCV的源码，可以知道具体有以下transform：
- 将图像大小调整为指定的尺寸（默认为224像素）；
- 将图像的像素值从[0, 255]范围线性缩放到[0, 1]范围；
- 使用预设的均值和标准差对图像进行归一化处理；
- 将图像从HWC（高度x宽度x通道）格式转换为CHW格式；

训练模式中会额外增加：
- 图像被随机裁剪到32x32像素。在裁剪之前，图像的上下左右边界会各自填充4个像素；
- 以50%的概率对图像进行水平翻转，这有助于模型学习到更多样化的特征。

2. 通过[mindcv.data.create_loader](https://mindcv.readthedocs.io/en/latest/api/mindcv.data.html#mindcv.data.create_loader)函数，进行数据转换和batch切分加载，我们需要将[create_transforms](https://mindcv.readthedocs.io/en/latest/api/mindcv.data.html#mindcv.data.create_transforms)返回的transform_list传入。

In [3]:
# 执行数据增强操作，生成所需数据集。
loader_train = create_loader(dataset=dataset_train,
                             batch_size=64,
                             is_training=True,
                             num_classes=num_classes,
                             transform=trans,
                             num_parallel_workers=num_workers)

num_batches = loader_train.get_dataset_size()

### 2.3 模型创建和加载

使用[create_model](https://mindcv.readthedocs.io/en/latest/api/mindcv.models.html#mindcv.models.create_model)接口获得实例化的DenseNet，并加载预训练权重densenet_121_224.ckpt（ImageNet数据集训练得到）。

In [4]:
from mindcv.models import create_model

# 实例化 DenseNet-121 模型并加载预训练权重。
network = create_model(model_name='densenet121', num_classes=num_classes, pretrained=True)

[WARNING] ME(6652:281473201414160,MainProcess):2024-01-23-04:40:36.942.49 [mindspore/train/serialization.py:1317] For 'load_param_into_net', 2 parameters in the 'net' are not loaded, because they are not in the 'parameter_dict', please check whether the network structure is consistent when training and loading checkpoint.
[WARNING] ME(6652:281473201414160,MainProcess):2024-01-23-04:40:36.104.151 [mindspore/train/serialization.py:1322] classifier.weight is not loaded.
[WARNING] ME(6652:281473201414160,MainProcess):2024-01-23-04:40:36.106.891 [mindspore/train/serialization.py:1322] classifier.bias is not loaded.


> 由于Cifar10和ImageNet数据集所需类别数量不同，分类器参数无法共享，出现分类器参数无法加载的告警不影响微调。

### 2.4 模型训练

通过[create_loss](https://mindcv.readthedocs.io/en/latest/api/mindcv.loss.html#mindcv.loss.create_loss)接口获得损失函数。

In [5]:
from mindcv.loss import create_loss

loss = create_loss(name='CE')

使用[create_scheduler](https://mindcv.readthedocs.io/en/latest/api/mindcv.scheduler.html#mindcv.scheduler.create_scheduler)接口设置学习率策略。

In [6]:
from mindcv.scheduler import create_scheduler

# 设置学习率策略
lr_scheduler = create_scheduler(steps_per_epoch=num_batches,
                                scheduler='constant',
                                lr=0.0001)

使用[create_optimizer](https://mindcv.readthedocs.io/en/latest/api/mindcv.optim.html#mindcv.optim.create_optimizer)接口创建优化器。

In [7]:
from mindcv.optim import create_optimizer

# 设置优化器
opt = create_optimizer(network.trainable_params(), opt='adam', lr=lr_scheduler) 

使用[mindspore.Model](https://mindspore.cn/docs/zh-CN/r1.8/api_python/mindspore/mindspore.Model.html)接口根据用户传入的参数封装可训练的实例。

In [8]:
from mindspore import Model

# 封装可训练或推理的实例
model = Model(network, loss_fn=loss, optimizer=opt, metrics={'accuracy'})

使用[`mindspore.Model.train`](https://mindspore.cn/docs/zh-CN/r1.8/api_python/mindspore/mindspore.Model.html#mindspore.Model.train)接口进行模型训练。

In [9]:
from mindspore import LossMonitor, TimeMonitor, CheckpointConfig, ModelCheckpoint

# 设置在训练过程中保存网络参数的回调函数
ckpt_save_dir = './ckpt' 
ckpt_config = CheckpointConfig(save_checkpoint_steps=num_batches)
ckpt_cb = ModelCheckpoint(prefix='densenet121-cifar10',
                          directory=ckpt_save_dir,
                          config=ckpt_config)

model.train(5, loader_train, callbacks=[LossMonitor(num_batches//5), TimeMonitor(num_batches//5), ckpt_cb], dataset_sink_mode=False)

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/ipykernel/iostream.py:153: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0xfffe701f4640>
  self._event_pipes[threading.current_thread()] = event_pipe


epoch: 1 step: 156, loss is 2.080545425415039
epoch: 1 step: 312, loss is 1.1506786346435547
epoch: 1 step: 468, loss is 0.7003673911094666
epoch: 1 step: 624, loss is 0.2888652980327606
epoch: 1 step: 780, loss is 0.21294066309928894
Train epoch time: 1143908.666 ms, per step time: 1462.799 ms
epoch: 2 step: 154, loss is 0.37496501207351685
epoch: 2 step: 310, loss is 0.3438137173652649
epoch: 2 step: 466, loss is 0.21980619430541992
epoch: 2 step: 622, loss is 0.19031941890716553
epoch: 2 step: 778, loss is 0.2117447406053543
Train epoch time: 558855.600 ms, per step time: 714.649 ms
epoch: 3 step: 152, loss is 0.04919916018843651
epoch: 3 step: 308, loss is 0.11679410189390182
epoch: 3 step: 464, loss is 0.10938809812068939
epoch: 3 step: 620, loss is 0.12382559478282928
epoch: 3 step: 776, loss is 0.04832660034298897
Train epoch time: 559823.761 ms, per step time: 715.887 ms
epoch: 4 step: 150, loss is 0.08343680202960968
epoch: 4 step: 306, loss is 0.022771470248699188
epoch: 4 st

### 2.5 模型效果评估

In [10]:
# 加载验证数据集
dataset_val = create_dataset(name='cifar10', root=cifar10_dir, split='test', shuffle=True, num_parallel_workers=num_workers, download=download)

# 执行数据增强操作，生成所需数据集。
loader_val = create_loader(dataset=dataset_val,
                           batch_size=64,
                           is_training=False,
                           num_classes=num_classes,
                           transform=trans,
                           num_parallel_workers=num_workers)

170052608B [20:22, 139101.36B/s]                                 


加载微调后的参数文件（densenet121-cifar10-5_782.ckpt）到模型。

根据用户传入的参数封装可推理的实例，加载验证数据集，验证微调的 DenseNet121模型精度。

In [11]:
# 验证微调后的DenseNet121的精度
acc = model.eval(loader_val, dataset_sink_mode=False)
print(acc)

{'accuracy': 0.9591}


## 3 总结

本案例基于MindSpore框架针对CIFAR-10数据集。首先介绍了Densenet网络的结构与特点，之后结合MindCV源码进行深入讲解。案例实现部分调用MindCV接口快速进行了CIFAR-10数据集的读取与处理，实例化Densenet121模型，然后训练以及评估。通过此案例进一步加深了对Densenet模型结构和特性的理解，并通过使用MindSpore框架和MindCV组件高效地完成了整个案例实现流程，加深了对框架提供的API的理解与掌握。